In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import matplotlib
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import precision_score,recall_score,f1_score,cohen_kappa_score
import statsmodels.api as sm
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import cross_val_score
import pylab as pl
pd.options.display.max_rows=None
pd.options.display.max_columns=None

In [2]:
df = pd.read_excel('transformed_df.xlsx')

In [3]:
df.head(2)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,Year,Month
0,343481,NaN,NaN,71.310442,16000,16000.0,36 months,2.794780,18.386629,D,D2,Manager,10+ years,RENT,76.282376,Source Verified,2018-04-01,0,n,NaN,NaN,debt_consolidation,Debt consolidation,482xx,MI,12.97,1,Sep-2006,1,20.0,NaN,3.493362,0,21.086054,4.0,5.252012,w,77.234847,12782.62,15.828322,5222.14,12.196131,28.918987,0.0,0.0,0.0,Feb-2019,4.563472,Mar-2019,Feb-2019,1,20.0,1,Joint App,155000.0,25.71,Source Verified,0,0,43.413863,3,1,2,3,2.0,65.932533,100.0,1,5,22.451932,51.0,37.714065,2,4,2,3.085858,7084.0,10763.0,7.2,0,0,137.0,14.460281,1.077967,1.068123,2,2.0,NaN,2.0,NaN,1,1.567930,2.060831,1.513386,2.028252,2.539273,2.900628,3.205608,2.088951,3.487390,0.0,0,1,1.427368,96.0,0.0,0,0,35.191359,54.301766,60.262431,90.793486,38618.0,Mar-2007,1.0,1.0,20.0,52.2,5.0,16.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2018,4
1,343482,NaN,NaN,87.134314,28675,28675.0,60 months,3.107894,21.092636,E,E5,officer,10+ years,MORTGAGE,74.441265,Verified,2018-04-01,0,n,NaN,NaN,debt_consolidation,Debt consolidation,114xx,NY,17.00,0,Jul-2005,1,NaN,NaN,4.147580,0,54.144615,69.4,7.973929,w,100.229002,26107.93,17.444102,8674.05,11.711664,41.597367,0.0,0.0,0.0,Feb-2019,4.757724,Mar-2019,Feb-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,61.236392,1,15,0,0,36.0,112.009436,116.0,2,4,82.265507,105.0,40.381219,0,1,1,1.969564,23197.0,6014.0,79.5,0,0,153.0,13.939855,1.077967,1.068123,2,2.0,NaN,0.0,NaN,0,1.857586,2.447468,1.782384,2.954972,5.117636,2.296616,3.286357,2.486553,4.139374,0.0,0,0,1.124303,100.0,75.0,0,0,45.366029,91.225286,82.316757,156.824990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [4]:
df.isnull().sum()

Unnamed: 0                                         0
id                                            150792
member_id                                     150792
loan_amnt                                          0
funded_amnt                                        0
funded_amnt_inv                                    0
term                                               0
int_rate                                           0
installment                                        0
grade                                              0
sub_grade                                          0
emp_title                                      13107
emp_length                                     12866
home_ownership                                     0
annual_inc                                         0
verification_status                                0
issue_d                                            0
loan_status                                        0
pymnt_plan                                    

In [5]:
df.shape

(150792, 148)

In [6]:
df=df[['Unnamed: 0',
'loan_amnt',
'term',
'int_rate',
'installment',
'sub_grade',
'home_ownership',
'annual_inc',
'verification_status',
'purpose',
'delinq_2yrs',
'open_acc',
'pub_rec',
'revol_bal',
'total_acc',
'initial_list_status',
'out_prncp',
'total_pymnt',
'total_rec_prncp',
'total_rec_int',
'total_rec_late_fee',
'last_pymnt_amnt',
'application_type',
'tot_coll_amt',
'tot_cur_bal',
'open_acc_6m',
'open_act_il',
'open_il_12m',
'open_il_24m',
'total_bal_il',
'open_rv_12m',
'open_rv_24m',
'max_bal_bc',
'total_rev_hi_lim',
'inq_fi',
'total_cu_tl',
'inq_last_12m',
'acc_open_past_24mths',
'mo_sin_old_rev_tl_op',
'mo_sin_rcnt_rev_tl_op',
'mo_sin_rcnt_tl',
'mort_acc',
'num_accts_ever_120_pd',
'num_actv_bc_tl',
'num_actv_rev_tl',
'num_bc_sats',
'num_bc_tl',
'num_il_tl',
'num_op_rev_tl',
'num_rev_accts',
'num_rev_tl_bal_gt_0',
'num_sats',
'num_tl_op_past_12m',
'pub_rec_bankruptcies',
'tot_hi_cred_lim',
'total_bal_ex_mort',
'total_bc_limit',
'total_il_high_credit_limit',
'disbursement_method',
'loan_status',
'il_util',
'mths_since_recent_inq',
'mths_since_rcnt_il',
'mo_sin_old_il_acct',
'bc_util',
'percent_bc_gt_75',
'bc_open_to_buy',
'mths_since_recent_bc',
'dti',
'revol_util',
'all_util',
'avg_cur_bal',
'pct_tl_nvr_dlq']]

In [7]:
df= df.rename({'Unnamed: 0':'ID'}, axis=1)

In [8]:
df = df.set_index('ID')

In [9]:
df.head(2)

,loan_amnt,term,int_rate,installment,sub_grade,home_ownership,annual_inc,verification_status,purpose,delinq_2yrs,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,application_type,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,total_bal_il,open_rv_12m,open_rv_24m,max_bal_bc,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_op_past_12m,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,disbursement_method,loan_status,il_util,mths_since_recent_inq,mths_since_rcnt_il,mo_sin_old_il_acct,bc_util,percent_bc_gt_75,bc_open_to_buy,mths_since_recent_bc,dti,revol_util,all_util,avg_cur_bal,pct_tl_nvr_dlq
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
343481,71.310442,36 months,2.794780,18.386629,D2,RENT,76.282376,Source Verified,debt_consolidation,1,3.493362,0,21.086054,5.252012,w,77.234847,15.828322,12.196131,28.918987,0.0,4.563472,Joint App,0,43.413863,3,1,2,3,65.932533,1,5,22.451932,37.714065,2,4,2,3.085858,14.460281,1.077967,1.068123,2,1,1.567930,2.060831,1.513386,2.028252,2.539273,2.900628,3.205608,2.088951,3.487390,1.427368,0,35.191359,54.301766,60.262431,90.793486,Cash,0,100.0,2.0,2.0,137.0,7.2,0.0,10763.0,2.0,12.97,4.0,51.0,7084.0,96.0
343482,87.134314,60 months,3.107894,21.092636,E5,MORTGAGE,74.441265,Verified,debt_consolidation,0,4.147580,0,54.144615,7.973929,w,100.229002,17.444102,11.711664,41.597367,0.0,4.757724,Individual,0,61.236392,1,15,0,0,112.009436,2,4,82.265507,40.381219,0,1,1,1.969564,13.939855,1.077967,1.068123,2,0,1.857586,2.447468,1.782384,2.954972,5.117636,2.296616,3.286357,2.486553,4.139374,1.124303,0,45.366029,91.225286,82.316757,156.824990,Cash,0,116.0,0.0,36.0,153.0,79.5,75.0,6014.0,2.0,17.00,69.4,105.0,23197.0,100.0


In [10]:
df.shape

(150792, 72)

In [11]:
df.isnull().sum()

loan_amnt                         0
term                              0
int_rate                          0
installment                       0
sub_grade                         0
home_ownership                    0
annual_inc                        0
verification_status               0
purpose                           0
delinq_2yrs                       0
open_acc                          0
pub_rec                           0
revol_bal                         0
total_acc                         0
initial_list_status               0
out_prncp                         0
total_pymnt                       0
total_rec_prncp                   0
total_rec_int                     0
total_rec_late_fee                0
last_pymnt_amnt                   0
application_type                  0
tot_coll_amt                      0
tot_cur_bal                       0
open_acc_6m                       0
open_act_il                       0
open_il_12m                       0
open_il_24m                 

In [12]:
df_num = df.select_dtypes(include=np.number)

In [13]:
df_num.shape

(150792, 64)

In [14]:
df_num.head(2)

,loan_amnt,int_rate,installment,annual_inc,delinq_2yrs,open_acc,pub_rec,revol_bal,total_acc,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,total_bal_il,open_rv_12m,open_rv_24m,max_bal_bc,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_op_past_12m,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,loan_status,il_util,mths_since_recent_inq,mths_since_rcnt_il,mo_sin_old_il_acct,bc_util,percent_bc_gt_75,bc_open_to_buy,mths_since_recent_bc,dti,revol_util,all_util,avg_cur_bal,pct_tl_nvr_dlq
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
343481,71.310442,2.794780,18.386629,76.282376,1,3.493362,0,21.086054,5.252012,77.234847,15.828322,12.196131,28.918987,0.0,4.563472,0,43.413863,3,1,2,3,65.932533,1,5,22.451932,37.714065,2,4,2,3.085858,14.460281,1.077967,1.068123,2,1,1.567930,2.060831,1.513386,2.028252,2.539273,2.900628,3.205608,2.088951,3.487390,1.427368,0,35.191359,54.301766,60.262431,90.793486,0,100.0,2.0,2.0,137.0,7.2,0.0,10763.0,2.0,12.97,4.0,51.0,7084.0,96.0
343482,87.134314,3.107894,21.092636,74.441265,0,4.147580,0,54.144615,7.973929,100.229002,17.444102,11.711664,41.597367,0.0,4.757724,0,61.236392,1,15,0,0,112.009436,2,4,82.265507,40.381219,0,1,1,1.969564,13.939855,1.077967,1.068123,2,0,1.857586,2.447468,1.782384,2.954972,5.117636,2.296616,3.286357,2.486553,4.139374,1.124303,0,45.366029,91.225286,82.316757,156.824990,0,116.0,0.0,36.0,153.0,79.5,75.0,6014.0,2.0,17.00,69.4,105.0,23197.0,100.0


In [15]:
df_cat = df.select_dtypes(exclude=np.number)

In [16]:
df_cat.shape

(150792, 8)

In [17]:
df_cat.head(2)

,term,sub_grade,home_ownership,verification_status,purpose,initial_list_status,application_type,disbursement_method
ID,,,,,,,,
343481,36 months,D2,RENT,Source Verified,debt_consolidation,w,Joint App,Cash
343482,60 months,E5,MORTGAGE,Verified,debt_consolidation,w,Individual,Cash


In [18]:
df_cat.isnull().sum()

term                   0
sub_grade              0
home_ownership         0
verification_status    0
purpose                0
initial_list_status    0
application_type       0
disbursement_method    0
dtype: int64

In [19]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors= 5)
imputed = imputer.fit_transform(df_num)
df_imputed = pd.DataFrame(imputed, columns=df_num.columns)

In [20]:
df_imputed.to_csv('knn_imputed_data.csv')

In [21]:
df_imputed = pd.get_dummies(df_imputed, drop_first =True)

In [22]:
X=df_imputed.drop('loan_status',axis=1)

In [23]:
y = df_imputed[['loan_status']]

In [25]:
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

#### Model 3.4 - NaiveBayes 

In [26]:
gnb = GaussianNB()
sc = StandardScaler()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= .30,random_state=10)
scaledXtrain = sc.fit_transform(Xtrain)
scaledXtest = sc.transform(Xtest)
gnb.fit(scaledXtrain,ytrain)
print("Training Accuracy")
print(gnb.score(Xtrain,ytrain))
print("Testing Accuracy")
print(gnb.score(scaledXtest,ytest))
scoresgnb = cross_val_score(gnb, scaledXtrain,ytrain, cv=10, scoring='roc_auc')
print("Cross Validation Accuracy")
print(np.mean(scoresgnb))
predicted_gnb = gnb.predict(scaledXtest)
print(confusion_matrix(ytest,predicted_gnb))
print(classification_report(ytest,predicted_gnb))

Training Accuracy
0.9379938230668663
Testing Accuracy
0.8323533312701711
Cross Validation Accuracy
0.7730631466896353
[[36143  6337]
 [ 1247  1511]]
              precision    recall  f1-score   support

         0.0       0.97      0.85      0.91     42480
         1.0       0.19      0.55      0.28      2758

    accuracy                           0.83     45238
   macro avg       0.58      0.70      0.59     45238
weighted avg       0.92      0.83      0.87     45238



#### Model 3.6 GradientBoosting  

In [27]:
gbc = GradientBoostingClassifier()
sc = StandardScaler()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= .30,random_state=10)
scaledXtrain = sc.fit_transform(Xtrain)
scaledXtest = sc.transform(Xtest)
gbc.fit(scaledXtrain,ytrain)
print("Training Accuracy")
print(gbc.score(Xtrain,ytrain))
print("Testing Accuracy")
print(gbc.score(scaledXtest,ytest))
scoresgbc = cross_val_score(gbc, scaledXtrain,ytrain, cv=10, scoring='roc_auc')
print("Cross Validation Accuracy")
print(np.mean(scoresgbc))
predicted_gbc = gbc.predict(scaledXtest)
print(confusion_matrix(ytest,predicted_gbc))
print(classification_report(ytest,predicted_gbc))

Training Accuracy
0.13172404645963204
Testing Accuracy
0.976524161103497
Cross Validation Accuracy
0.9482441744149362
[[42426    54]
 [ 1008  1750]]
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     42480
         1.0       0.97      0.63      0.77      2758

    accuracy                           0.98     45238
   macro avg       0.97      0.82      0.88     45238
weighted avg       0.98      0.98      0.97     45238



In [28]:
xgb = XGBClassifier()
sc = StandardScaler()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= .30,random_state=10)
scaledXtrain = sc.fit_transform(Xtrain)
scaledXtest = sc.transform(Xtest)
xgb.fit(scaledXtrain,ytrain)
print("Training Accuracy")
print(xgb.score(Xtrain,ytrain))
print("Testing Accuracy")
print(xgb.score(scaledXtest,ytest))
scoresxgb = cross_val_score(xgb, scaledXtrain,ytrain, cv=10, scoring='roc_auc')
print("Cross Validation Accuracy")
print(np.mean(scoresxgb))
predicted_xgb = xgb.predict(scaledXtest)
print(confusion_matrix(ytest,predicted_xgb))
print(classification_report(ytest,predicted_xgb))

[01:26:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training Accuracy
0.9350000947382382
Testing Accuracy
0.9808125911844024
[01:27:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:27:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavio